In [126]:
import snscrape.modules.twitter as sntwitter
from datetime import datetime
import time
import random
import pandas as pd
import nltk
from dotenv import load_dotenv
import os
import re


In [127]:
twitter_username = os.getenv("TWT_USERNAME")
twitter_password = os.getenv("TWT_PASSWORD")



In [114]:
twitter_user = sntwitter.TwitterUserScraper("astrologelvan")
tweets = []
for i, tweet in enumerate(twitter_user.get_items()):
    if i == 1000:
        break

    tweets.append(tweet)

In [128]:
df = pd.DataFrame(tweets)
df.head()

,url,date,content,renderedContent,id,user,replyCount,retweetCount,likeCount,quoteCount,...,media,retweetedTweet,quotedTweet,inReplyToTweetId,inReplyToUser,mentionedUsers,coordinates,place,hashtags,cashtags
0,https://twitter.com/astrologelvan/status/16114...,2023-01-06 20:33:01+00:00,Papa nereye ben oraya.,Papa nereye ben oraya.,1611460851554197549,"{'username': 'astrologelvan', 'id': 144486876,...",6,1,131,2,...,None,None,None,NaN,None,None,None,None,None,None
1,https://twitter.com/astrologelvan/status/16113...,2023-01-06 11:54:46+00:00,@berffhelloo türklerin medenileri,@berffhelloo türklerin medenileri,1611330429386121220,"{'username': 'astrologelvan', 'id': 144486876,...",0,1,8,1,...,None,None,None,1.611112e+18,"{'username': 'berffhelloo', 'id': 158783263287...","[{'username': 'berffhelloo', 'id': 15878326328...",None,None,None,None
2,https://twitter.com/astrologelvan/status/16111...,2023-01-06 01:37:44+00:00,@Missgadgett ahhah yazık be,@Missgadgett ahhah yazık be,1611175150170374144,"{'username': 'astrologelvan', 'id': 144486876,...",0,0,7,0,...,None,None,None,1.611120e+18,"{'username': 'Missgadgett', 'id': 157437050142...","[{'username': 'Missgadgett', 'id': 15743705014...",None,None,None,None
3,https://twitter.com/astrologelvan/status/16111...,2023-01-06 01:37:01+00:00,@are_if_astro hahhahahah,@are_if_astro hahhahahah,1611174969576411137,"{'username': 'astrologelvan', 'id': 144486876,...",0,0,6,0,...,None,None,None,1.611126e+18,"{'username': 'are_if_astro', 'id': 3042974941,...","[{'username': 'are_if_astro', 'id': 3042974941...",None,None,None,None
4,https://twitter.com/astrologelvan/status/16111...,2023-01-06 01:36:31+00:00,@zeyyneepkrks hahaahhaa kız yıka da çıkart.,@zeyyneepkrks hahaahhaa kız yıka da çıkart.,1611174843587649537,"{'username': 'astrologelvan', 'id': 144486876,...",4,2,112,0,...,None,None,None,1.611174e+18,"{'username': 'zeyyneepkrks', 'id': 12086977091...","[{'username': 'zeyyneepkrks', 'id': 1208697709...",None,None,None,None


In [129]:
df.to_csv("astrologelvan.csv", index=False)

In [130]:
df = df[['content']]
df.to_csv('astrologelvan.csv',index=False)

In [131]:
import string

In [132]:
stop_words = sorted(["ile","ve","imza","bazında","bu","neydi","gelen","açıklamamız","en","son","yılı","rlerde","etlik",
"yeni","çok","daha","de","olan","da","gibi","aldık","yok","ama","çıkan","formayla","devam","başsağlığı",
"ediyor","ilk","hayırlı","dikkatine","arasında","ilgili","sn","hastanesi","beyaz","günü","olarak",
"içinde","yeni","rlerde","er","her","göre","dün","büyük","bazı","maçların","taraftarları","var",
"tüm","döner","ediyoruz","üzere","olmak","bugün","bin","başta",])

In [133]:
def clean_text(text):
    text = re.sub("https?://\S+", "", text)
    text = text.replace("\\n", " ")
    text = re.sub("[0-9]+", "", text)
    text = text.translate(str.maketrans("", "", string.punctuation))
    text = re.sub("’|“|”|‘|–|—", "", text)
    text = re.sub(" 🇹🇷", "", text)
    text = [t for t in text.split() if t not in stop_words]
    return " ".join(text)

In [134]:
df["clean"] = df.apply(lambda row: clean_text(row["content"]), axis=1)


In [135]:
df["clean"][:10]


0                                Papa nereye ben oraya
1                     berffhelloo türklerin medenileri
2                           Missgadgett ahhah yazık be
3                                areifastro hahhahahah
4               zeyyneepkrks hahaahhaa kız yıka çıkart
5    İtalyada dateime çıktım Sonra başka bi İtalyan...
6                          Gökten yağmur herif yağacak
7                                      CengizKARAN net
8                                   bettythefae aslan😅
9                         asparnews ahahha biliyorum😍😅
Name: clean, dtype: object

In [136]:
cleaned_df = df[['clean']]
cleaned_df.to_csv('astrologelvan_clean.csv',index=False)


In [137]:
cleaned_df.head()


,clean
0,Papa nereye ben oraya
1,berffhelloo türklerin medenileri
2,Missgadgett ahhah yazık be
3,areifastro hahhahahah
4,zeyyneepkrks hahaahhaa kız yıka çıkart


In [138]:
df = pd.read_csv('astrologelvan_clean.csv')

df.dropna(subset=['clean'], inplace=True)

df['tokens'] = df['clean'].apply(nltk.word_tokenize)

df['bigrams'] = df['tokens'].apply(lambda x: list(nltk.bigrams(x)))

bigram_freq = pd.Series([bigram for bigrams in df['bigrams'] for bigram in bigrams]).value_counts()

print(bigram_freq.head(30))

(Bu, hafta)            20
(o, kadar)             16
(burcu, haftanın)      13
(Merkür, retrosu)      12
(ne, kadar)            12
(bir, dönem)           11
(ne, varsa)            11
(dikkat, et)           10
(Yeni, bir)            10
(hazır, ol)            10
(Bu, gece)             10
(O, yüzden)             8
(pat, diye)             8
(bir, gün)              8
(iyi, gelecek)          8
(bir, şey)              7
(Mars, İkizler)         7
(Her, şey)              7
(Geliyor, gelmekte)     6
(yasın, evreleri)       6
(burcunda, yasın)       6
(bir, hafta)            6
(ne, ise)               6
(cinnet, şeyi)          6
(Mars, İkizlerde)       5
(koparma, ritüeli)      5
(ballı, çöreğim)        5
(Astrolog, Elvan)       5
(Asıl, mesele)          5
(ay, boyunca)           5
dtype: int64
